# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

## Задание 1. 

### Обязательная часть

Будем парсить страницу со свежеми новостям на [habr.com/ru/all/](https://habr.com/ru/all/).

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

`KEYWORDS = ['python', 'парсинг']`

 Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы). 
 
В итоге должен формироваться датафрейм вида: `<дата> - <заголовок> - <ссылка>`

### Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.  

Итоговый датафрейм формировать со столбцами: `<дата> - <заголовок> - <ссылка> - <текст_статьи>`


### Решение

In [124]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

KEYWORDS = ['python', 'парсинг']

# получаем страницу с самыми свежими постами
req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(req.text, 'html.parser')

# извлекаем ссылки на посты
post_links = list(map(lambda x: x.find('a', class_='post__title_link').get('href'), soup.find_all('article', class_='post')))

# Подготавливаем датафрейм
res = pd.DataFrame(columns=['post_dt', 'header', 'link', 'body'])

# Проходимся по ссылкам
for link in post_links:
    # Собираем необходимые данные со страницы
    page = BeautifulSoup(requests.get(link).text, 'html.parser')
    header = page.find('span', class_='post__title-text').text
    post_dt = datetime.strptime(page.find('span', class_='post__time').get('data-time_published'), '%Y-%m-%dT%H:%MZ')
    body = page.find('div', id='post-content-body').text
    # Проверяем подходят ли нам по кейвордам и если подходят добавляем в дф-результат
    for kv in KEYWORDS:
        if kv in header or kv in body:
            res = res.append({x:eval(x) for x in ['post_dt', 'header', 'link', 'body']}, ignore_index=True)

res

# Единственное что в текстах теряются абзацы :(

,post_dt,header,link,body
0,2021-04-05 11:19:00,Полезности для разработчика на Django,https://habr.com/ru/post/550784/,ПредисловиеДля написания данной статьи был изу...
1,2021-04-05 10:47:00,Magento 2 UI Components. Часть 1: общее устрой...,https://habr.com/ru/company/rshb/blog/550772/,Привет! Меня зовут Павел и я Magento 2 бэкенд-...
2,2021-04-05 10:45:00,Impala для Python-разработчика на примере опре...,https://habr.com/ru/company/wargaming/blog/550...,"Всем привет.Как известно, есть множество разли..."


## Задание 2.

### Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса [Avast Hack Ckeck](https://www.avast.com/hackcheck/).
Список email-ов задаем переменной в начале кода:  
`EMAIL = [xxx@x.ru, yyy@y.com]`

В итоге должен формироваться датафрейм со столбцами: `<почта> - <дата утечки> - <источник утечки> - <описание утечки>`  

**Подсказка**: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

### Решение

In [125]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import json
import re

EMAIL = ['xxx@xx.ru', 'yyy@yy.com']

URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'

params = {
    'emailAddresses': EMAIL
}
headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36'
    ,'Vaar-Header-App-Build-Version': '1.0.0'
    ,'Vaar-Version': '0'
    ,'Vaar-Header-App-Product-Name': 'hackcheck-web-avast'
}

req = requests.post(URL, headers=headers, json=params)

soup = BeautifulSoup(req.content, 'html.parser')

dres = json.loads(soup.text)

# Подготавливаем датафрейм
res = pd.DataFrame(columns=['email', 'dt_leaks', 'source_leaks', 'dt_published',  'description'])

e2l = {}
# Определяем какие имейлы были в каких утечках
for k, v in dres['summary'].items():
    e2l[k] = v['breaches']

    
# Смотрим сливы
for d in dres['breaches'].values():
    try:
        dt_published = datetime.strptime(d['publishDate'], '%Y-%m-%dT%H:%M:%SZ')
        source_leaks = d['site']
        description = d['description']
        # доставая дату из текста описания утечки получается довольно неоднозначно.
        # Так что либо в задании имелась ввиду дата публикации, а не утечки, либо просто не получилось нормально достать дату утечки из статьи.
        dt_leaks = ' '.join(re.findall(r'[0-9]{4}|\b{Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec}*', description))
#         Смотрим к какому из исходных имейлов относятся сливы, фиксируем ответ
        for k,v in e2l.items():
            if d['breachId'] in v:
                email = k
                res = res.append({x:eval(x) for x in ['email', 'dt_leaks', 'source_leaks', 'dt_published', 'description']}, ignore_index=True)
    except:
        pass

res


,email,dt_leaks,source_leaks,dt_published,description
0,yyy@yy.com,Oct 2015,ir.netease.com,2016-11-07,"In October 2015, the Chinese internet and gami..."
1,yyy@yy.com,2015 2016,crackingforum.com,2017-01-31,"In 2015, hacker website CrackingForum.com's us..."
2,yyy@yy.com,Nov 2020,onlinepsychometrictest.com,2020-12-17,"In November 2020, a collection of over 23,000 ..."
3,yyy@yy.com,,hiapk.com,2017-09-07,Chinese technology site HIAPK.com's user datab...
4,xxx@xx.ru,,gamma.mmdm.ru,2021-01-07,At an unconfirmed date the home page for the R...
5,yyy@yy.com,,romwe.com,2020-04-02,"At an unconfirmed date, the online fashion ret..."
6,yyy@yy.com,2018,chegg.com,2018-08-24,"In 2018, education tech company Chegg's databa..."
7,yyy@yy.com,Jun 2020,wattpad.com,2020-07-23,"In June 2020, the online writing community Wat..."
8,yyy@yy.com,2016 Jun 2013,myspace.com,2016-10-21,"Shortly before the 2016 Memorial Day weekend, ..."
9,yyy@yy.com,,azcentral.com,2020-01-03,"At an unconfirmed date, online Arizona newspap..."


### Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.  
Документация к API VK: https://vk.com/dev/methods
, вам поможет метод [wall.get](https://vk.com/dev/wall.get)  
```
GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ  
```

В итоге должен формироваться датафрейм со столбцами: `<дата поста> - <текст поста>`

### Решение

In [126]:
import requests
import pandas as pd
from datetime import datetime

access_token = ''

# Не сразу увидела, что в метод wall.get можно передавать алиас сообщества/группы, поэтому сначала доставала ид по алиасу отдельным действием
params = {
    'group_id':'netology'
    , 'fields':'owner_id'
    , 'access_token':access_token
    , 'v':'5.103'
}

# Получаем ид сообщества
req = requests.get('https://api.vk.com/method/groups.getById', params)
group_id = req.json()['response'][0]['id']

# Подготавливаем датафрейм
res = pd.DataFrame(columns=['dt_published',  'post_text'])

# забираем записи стены
params = {
    'owner_id':'-' + str(group_id)
    , 'fields':'owner_id'
    , 'access_token':access_token
    , 'v':'5.103'
    , 'domain':'netology'
    , 'count':50
}
req = requests.get('https://api.vk.com/method/wall.get', params)

for x in req.json()['response']['items']:
    dt_published = datetime.fromtimestamp(x['date'])
    post_text = x['text']
    res = res.append({x:eval(x) for x in ['dt_published', 'post_text']}, ignore_index=True)

    
res

,dt_published,post_text
0,2021-04-02 10:18:00,"Пишем коды для веб-страниц, осваиваем языки пр..."
1,2021-04-05 10:16:00,🚀 Запустили бесплатный курс «Основы интернет-м...
2,2021-04-04 13:52:00,"Мы знаем, что найти работу не всегда легко 🤗 \..."
3,2021-04-03 15:25:00,*партнёрский пост* \n \n15 апреля состоится он...
4,2021-04-03 13:46:00,"Чтобы писать красиво, просто и понятно — недос..."
5,2021-04-01 18:46:00,Прокрастинация — это когда нужно работать здес...
6,2021-04-01 11:15:00,"Чтобы сменить специальность, нужно признать, ч..."
7,2021-03-31 15:32:32,Разыгрываем 10 доступов к библиотеке видеокурс...
8,2021-03-31 10:49:00,"Кажется, в SMM всё просто: написал пост, ответ..."
9,2021-03-30 17:49:00,В 2020 году сербская художница-перформанистка ...
